In [1]:
import pyspark
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# Read Case entity 

In [18]:
cases = spark.read.load("coronavirusdataset/Case.csv",format="csv", sep=",", inferSchema="true", header="true")

In [3]:
cases.show(5)

+-------+--------+-------------+-----+--------------------+---------+---------+----------+
|case_id|province|         city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+-------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|   Yongsan-gu| true|       Itaewon Clubs|       72|37.538621|126.992652|
|1000002|   Seoul|      Guro-gu| true| Guro-gu Call Center|       98|37.508163|126.884387|
|1000003|   Seoul|Dongdaemun-gu| true|       Dongan Church|       20|37.592888|127.056766|
|1000004|   Seoul|      Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000005|   Seoul| Eunpyeong-gu| true|Eunpyeong St. Mar...|       14| 37.63369|  126.9165|
+-------+--------+-------------+-----+--------------------+---------+---------+----------+
only showing top 5 rows



In [4]:
cases.columns

['case_id',
 'province',
 'city',
 'group',
 'infection_case',
 'confirmed',
 'latitude',
 'longitude']

# rdd to Pandas

In [5]:
cases.limit(10).toPandas()

,case_id,province,city,group,infection_case,confirmed,latitude,longitude
0,1000001,Seoul,Yongsan-gu,True,Itaewon Clubs,72,37.538621,126.992652
1,1000002,Seoul,Guro-gu,True,Guro-gu Call Center,98,37.508163,126.884387
2,1000003,Seoul,Dongdaemun-gu,True,Dongan Church,20,37.592888,127.056766
3,1000004,Seoul,Guro-gu,True,Manmin Central Church,41,37.481059,126.894343
4,1000005,Seoul,Eunpyeong-gu,True,Eunpyeong St. Mary's Hospital,14,37.63369,126.9165
5,1000006,Seoul,Seongdong-gu,True,Seongdong-gu APT,13,37.55713,127.0403
6,1000007,Seoul,Jongno-gu,True,Jongno Community Center,10,37.57681,127.006
7,1000008,Seoul,Jung-gu,True,Jung-gu Fashion Company,7,37.562405,126.984377
8,1000009,Seoul,from other city,True,Shincheonji Church,8,-,-
9,1000010,Seoul,-,False,overseas inflow,321,-,-


## Changing the Name of columns

- rdd.withColumnRenamed("A","B")

In [6]:
cases = cases.withColumnRenamed("infection_case",'infection_source')

In [7]:
cases.show(5)

+-------+--------+-------------+-----+--------------------+---------+---------+----------+
|case_id|province|         city|group|    infection_source|confirmed| latitude| longitude|
+-------+--------+-------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|   Yongsan-gu| true|       Itaewon Clubs|       72|37.538621|126.992652|
|1000002|   Seoul|      Guro-gu| true| Guro-gu Call Center|       98|37.508163|126.884387|
|1000003|   Seoul|Dongdaemun-gu| true|       Dongan Church|       20|37.592888|127.056766|
|1000004|   Seoul|      Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000005|   Seoul| Eunpyeong-gu| true|Eunpyeong St. Mar...|       14| 37.63369|  126.9165|
+-------+--------+-------------+-----+--------------------+---------+---------+----------+
only showing top 5 rows



In [8]:
cases = cases.toDF(*['case_id','province','city','group','infection_source','confirmed','latitude','longitude'])

In [9]:
cases.show(5)

+-------+--------+-------------+-----+--------------------+---------+---------+----------+
|case_id|province|         city|group|    infection_source|confirmed| latitude| longitude|
+-------+--------+-------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|   Yongsan-gu| true|       Itaewon Clubs|       72|37.538621|126.992652|
|1000002|   Seoul|      Guro-gu| true| Guro-gu Call Center|       98|37.508163|126.884387|
|1000003|   Seoul|Dongdaemun-gu| true|       Dongan Church|       20|37.592888|127.056766|
|1000004|   Seoul|      Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000005|   Seoul| Eunpyeong-gu| true|Eunpyeong St. Mar...|       14| 37.63369|  126.9165|
+-------+--------+-------------+-----+--------------------+---------+---------+----------+
only showing top 5 rows



- selecting features: rdd.select("col1",'col2',,,,)

In [10]:
cases = cases.select('province','city','infection_source')

In [11]:
cases.show(6)

+--------+-------------+--------------------+
|province|         city|    infection_source|
+--------+-------------+--------------------+
|   Seoul|   Yongsan-gu|       Itaewon Clubs|
|   Seoul|      Guro-gu| Guro-gu Call Center|
|   Seoul|Dongdaemun-gu|       Dongan Church|
|   Seoul|      Guro-gu|Manmin Central Ch...|
|   Seoul| Eunpyeong-gu|Eunpyeong St. Mar...|
|   Seoul| Seongdong-gu|    Seongdong-gu APT|
+--------+-------------+--------------------+
only showing top 6 rows



## Sort

- sorting values by columns

- rdd.sort("feature1" , ascending=[True|False]).show()

- df.sort(F.desc("feature1")).show()
- from pyspark.sql import functions as F

In [12]:
cases.sort("confirmed",ascending=False).show()

+-----------------+---------------+--------------------+
|         province|           city|    infection_source|
+-----------------+---------------+--------------------+
|            Daegu|         Nam-gu|  Shincheonji Church|
|            Daegu|              -|contact with patient|
|            Daegu|              -|                 etc|
| Gyeongsangbuk-do|from other city|  Shincheonji Church|
|            Seoul|              -|     overseas inflow|
|      Gyeonggi-do|              -|     overseas inflow|
|            Daegu|   Dalseong-gun|Second Mi-Ju Hosp...|
| Gyeongsangbuk-do|              -|contact with patient|
| Gyeongsangbuk-do|              -|                 etc|
|            Daegu|         Seo-gu|Hansarang Convale...|
| Gyeongsangbuk-do|   Cheongdo-gun|Cheongdo Daenam H...|
|Chungcheongnam-do|     Cheonan-si|gym facility in C...|
|            Daegu|   Dalseong-gun|Daesil Convalesce...|
|            Seoul|        Guro-gu| Guro-gu Call Center|
|            Seoul|     Yongsan

In [13]:
from pyspark.sql.types import DoubleType, IntegerType, StringType

In [14]:
from pyspark.sql import functions as F

In [15]:
cases.sort(F.desc("confirmed")).show()

+-----------------+---------------+--------------------+
|         province|           city|    infection_source|
+-----------------+---------------+--------------------+
|            Daegu|         Nam-gu|  Shincheonji Church|
|            Daegu|              -|contact with patient|
|            Daegu|              -|                 etc|
| Gyeongsangbuk-do|from other city|  Shincheonji Church|
|            Seoul|              -|     overseas inflow|
|      Gyeonggi-do|              -|     overseas inflow|
|            Daegu|   Dalseong-gun|Second Mi-Ju Hosp...|
| Gyeongsangbuk-do|              -|contact with patient|
| Gyeongsangbuk-do|              -|                 etc|
|            Daegu|         Seo-gu|Hansarang Convale...|
| Gyeongsangbuk-do|   Cheongdo-gun|Cheongdo Daenam H...|
|Chungcheongnam-do|     Cheonan-si|gym facility in C...|
|            Daegu|   Dalseong-gun|Daesil Convalesce...|
|            Seoul|        Guro-gu| Guro-gu Call Center|
|            Seoul|     Yongsan

## Casting 

In [16]:
# cases = cases.withColumn('confirmed',F.col("confirmed").cast(IntegerType()))
cases = cases.withColumn('city',F.col('city').cast(StringType()))

In [19]:
cases.select("confirmed").show()

+---------+
|confirmed|
+---------+
|       72|
|       98|
|       20|
|       41|
|       14|
|       13|
|       10|
|        7|
|        8|
|      321|
|       18|
|       24|
|       39|
|       12|
|        5|
|        6|
|        4|
|        1|
|       25|
|       18|
+---------+
only showing top 20 rows



## Filter

In [20]:
cases.filter((cases.confirmed>10) & (cases.province=='Daegu')).show()

+-------+--------+------------+-----+--------------------+---------+---------+----------+
|case_id|province|        city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+------------+-----+--------------------+---------+---------+----------+
|1200001|   Daegu|      Nam-gu| true|  Shincheonji Church|     4510| 35.84008|  128.5667|
|1200002|   Daegu|Dalseong-gun| true|Second Mi-Ju Hosp...|      196|35.857375|128.466651|
|1200003|   Daegu|      Seo-gu| true|Hansarang Convale...|      128|35.885592|128.556649|
|1200004|   Daegu|Dalseong-gun| true|Daesil Convalesce...|      100|35.857393|128.466653|
|1200005|   Daegu|     Dong-gu| true|     Fatima Hospital|       37| 35.88395|128.624059|
|1200007|   Daegu|           -|false|     overseas inflow|       24|        -|         -|
|1200008|   Daegu|           -|false|contact with patient|      929|        -|         -|
|1200009|   Daegu|           -|false|                 etc|      724|        -|         -|
+-------+-

- check for Seoul Province

In [21]:
cases.filter(cases.province=='Seoul').show()

+-------+--------+---------------+-----+--------------------+---------+---------+----------+
|case_id|province|           city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+---------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|     Yongsan-gu| true|       Itaewon Clubs|       72|37.538621|126.992652|
|1000002|   Seoul|        Guro-gu| true| Guro-gu Call Center|       98|37.508163|126.884387|
|1000003|   Seoul|  Dongdaemun-gu| true|       Dongan Church|       20|37.592888|127.056766|
|1000004|   Seoul|        Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000005|   Seoul|   Eunpyeong-gu| true|Eunpyeong St. Mar...|       14| 37.63369|  126.9165|
|1000006|   Seoul|   Seongdong-gu| true|    Seongdong-gu APT|       13| 37.55713|  127.0403|
|1000007|   Seoul|      Jongno-gu| true|Jongno Community ...|       10| 37.57681|   127.006|
|1000008|   Seoul|        Jung-gu| true|Jung-gu Fashion C...|        7

In [22]:
cases.filter((cases.confirmed>10) & (cases.province=='Seoul')).show()

+-------+--------+-------------+-----+--------------------+---------+---------+----------+
|case_id|province|         city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+-------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|   Yongsan-gu| true|       Itaewon Clubs|       72|37.538621|126.992652|
|1000002|   Seoul|      Guro-gu| true| Guro-gu Call Center|       98|37.508163|126.884387|
|1000003|   Seoul|Dongdaemun-gu| true|       Dongan Church|       20|37.592888|127.056766|
|1000004|   Seoul|      Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000005|   Seoul| Eunpyeong-gu| true|Eunpyeong St. Mar...|       14| 37.63369|  126.9165|
|1000006|   Seoul| Seongdong-gu| true|    Seongdong-gu APT|       13| 37.55713|  127.0403|
|1000010|   Seoul|            -|false|     overseas inflow|      321|        -|         -|
|1000011|   Seoul|            -|false|contact with patient|       18|        -|         -|

In [23]:
cases.filter((cases.confirmed>10) &( cases.province=='Seoul')).show(10)

+-------+--------+-------------+-----+--------------------+---------+---------+----------+
|case_id|province|         city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+-------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|   Yongsan-gu| true|       Itaewon Clubs|       72|37.538621|126.992652|
|1000002|   Seoul|      Guro-gu| true| Guro-gu Call Center|       98|37.508163|126.884387|
|1000003|   Seoul|Dongdaemun-gu| true|       Dongan Church|       20|37.592888|127.056766|
|1000004|   Seoul|      Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000005|   Seoul| Eunpyeong-gu| true|Eunpyeong St. Mar...|       14| 37.63369|  126.9165|
|1000006|   Seoul| Seongdong-gu| true|    Seongdong-gu APT|       13| 37.55713|  127.0403|
|1000010|   Seoul|            -|false|     overseas inflow|      321|        -|         -|
|1000011|   Seoul|            -|false|contact with patient|       18|        -|         -|

# GroupBy

In [24]:
cases.groupBy(['province']).agg(F.sum("confirmed")).show()

+-----------------+--------------+
|         province|sum(confirmed)|
+-----------------+--------------+
|           Sejong|            46|
|            Ulsan|            42|
|Chungcheongbuk-do|            38|
|       Gangwon-do|            51|
|          Gwangju|            30|
| Gyeongsangbuk-do|          1314|
|            Daegu|          6650|
| Gyeongsangnam-do|           115|
|          Incheon|            79|
|          Jeju-do|            14|
|      Gyeonggi-do|           564|
|            Busan|           139|
|          Daejeon|            34|
|            Seoul|           646|
|Chungcheongnam-do|           137|
|     Jeollabuk-do|            15|
|     Jeollanam-do|            16|
+-----------------+--------------+



In [25]:
cases.groupBy(['city']).agg(F.sum('confirmed')).show()

+---------------+--------------+
|           city|sum(confirmed)|
+---------------+--------------+
|     Cheonan-si|           103|
|from other city|           845|
|     Yongsan-gu|            72|
|        Dong-gu|            37|
|         Sejong|            38|
|       Wonju-si|             3|
|     Suyeong-gu|             5|
|   Geochang-gun|            18|
|  Dongdaemun-gu|            20|
|     Dongnae-gu|            39|
|         Jin-gu|             4|
|    Changwon-si|             7|
|         Nam-gu|          4510|
|   Gyeongsan-si|            99|
|      Jongno-gu|            10|
|    Bonghwa-gun|            68|
|   Cheongdo-gun|           120|
|              -|          3057|
|     Goesan-gun|            11|
|    Chilgok-gun|            36|
+---------------+--------------+
only showing top 20 rows



In [26]:
cases.groupBy(['province','city']).agg(F.sum("confirmed"),F.max("confirmed")).show()

+----------------+---------------+--------------+--------------+
|        province|           city|sum(confirmed)|max(confirmed)|
+----------------+---------------+--------------+--------------+
|Gyeongsangnam-do|       Jinju-si|            10|            10|
|           Seoul|        Guro-gu|           139|            98|
|         Daejeon|              -|            27|            10|
|    Jeollabuk-do|from other city|             1|             1|
|Gyeongsangnam-do|Changnyeong-gun|             7|             7|
|           Seoul|              -|           363|           321|
|         Jeju-do|from other city|             1|             1|
|Gyeongsangbuk-do|              -|           336|           192|
|Gyeongsangnam-do|   Geochang-gun|            18|            10|
|         Incheon|from other city|            22|            20|
|           Busan|              -|            72|            29|
|           Daegu|         Seo-gu|           128|           128|
|           Busan|     Su

In [27]:
cases.groupBy(['province','city']).agg(F.sum("confirmed").alias("totalconfirmed"),F.max("confirmed").alias("maxfROM")).show()

+----------------+---------------+--------------+-------+
|        province|           city|totalconfirmed|maxfROM|
+----------------+---------------+--------------+-------+
|Gyeongsangnam-do|       Jinju-si|            10|     10|
|           Seoul|        Guro-gu|           139|     98|
|         Daejeon|              -|            27|     10|
|    Jeollabuk-do|from other city|             1|      1|
|Gyeongsangnam-do|Changnyeong-gun|             7|      7|
|           Seoul|              -|           363|    321|
|         Jeju-do|from other city|             1|      1|
|Gyeongsangbuk-do|              -|           336|    192|
|Gyeongsangnam-do|   Geochang-gun|            18|     10|
|         Incheon|from other city|            22|     20|
|           Busan|              -|            72|     29|
|           Daegu|         Seo-gu|           128|    128|
|           Busan|     Suyeong-gu|             5|      5|
|     Gyeonggi-do|   Uijeongbu-si|            50|     50|
|           Se

## join

In [28]:
# cases = spark.read.load("coronavirusdataset/Case.csv",format="csv", sep=",", inferSchema="true", header="true")
regions = spark.read.load("coronavirusdataset/Region.csv",format='csv',sep=',',inferSchema='true',header='true')

In [29]:
regions.show()

+-----+--------+-------------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
| code|province|         city| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|
+-----+--------+-------------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|10000|   Seoul|        Seoul|37.566953|126.977977|                    607|               830|              48|         1.44|                   15.38|                5.8|             22739|
|10010|   Seoul|   Gangnam-gu|37.518421|127.047222|                     33|                38|               0|         4.18|                   13.17|                4.3|              3088|
|10020|   Seoul|  Gangdong-gu|37.530492|127.123837

In [30]:
regions.columns

['code',
 'province',
 'city',
 'latitude',
 'longitude',
 'elementary_school_count',
 'kindergarten_count',
 'university_count',
 'academy_ratio',
 'elderly_population_ratio',
 'elderly_alone_ratio',
 'nursing_home_count']

In [32]:
regions.select('province','city','academy_ratio','elementary_school_count').show()

+--------+-------------+-------------+-----------------------+
|province|         city|academy_ratio|elementary_school_count|
+--------+-------------+-------------+-----------------------+
|   Seoul|        Seoul|         1.44|                    607|
|   Seoul|   Gangnam-gu|         4.18|                     33|
|   Seoul|  Gangdong-gu|         1.54|                     27|
|   Seoul|   Gangbuk-gu|         0.67|                     14|
|   Seoul|   Gangseo-gu|         1.17|                     36|
|   Seoul|    Gwanak-gu|         0.89|                     22|
|   Seoul|  Gwangjin-gu|         1.16|                     22|
|   Seoul|      Guro-gu|          1.0|                     26|
|   Seoul| Geumcheon-gu|         0.96|                     18|
|   Seoul|     Nowon-gu|         1.39|                     42|
|   Seoul|    Dobong-gu|         0.95|                     23|
|   Seoul|Dongdaemun-gu|         1.06|                     21|
|   Seoul|   Dongjak-gu|         1.17|                 

In [33]:
regions.sort("academy_ratio").show()

+-----+-----------------+--------------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
| code|         province|          city| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|
+-----+-----------------+--------------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|14090|          Incheon|    Ongjin-gun|37.446777|126.636747|                      6|                10|               0|         0.19|                   25.35|               11.1|                32|
|51120|     Jeollanam-do|     Sinan-gun|34.833564|126.351636|                     19|                22|               0|          0.2|                   35.18|               21.0|                77|


In [35]:
join_test = cases.join(regions,['province','city'],how='left')
join_test.limit(10).toPandas()

,province,city,case_id,group,infection_case,confirmed,latitude,longitude,code,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,Seoul,Yongsan-gu,1000001,True,Itaewon Clubs,72,37.538621,126.992652,10210.0,37.532768,126.990021,15.0,13.0,1.0,0.68,16.87,6.5,435.0
1,Seoul,Guro-gu,1000002,True,Guro-gu Call Center,98,37.508163,126.884387,10070.0,37.495632,126.887650,26.0,34.0,3.0,1.00,16.21,5.7,741.0
2,Seoul,Dongdaemun-gu,1000003,True,Dongan Church,20,37.592888,127.056766,10110.0,37.574552,127.039721,21.0,31.0,4.0,1.06,17.26,6.7,832.0
3,Seoul,Guro-gu,1000004,True,Manmin Central Church,41,37.481059,126.894343,10070.0,37.495632,126.887650,26.0,34.0,3.0,1.00,16.21,5.7,741.0
4,Seoul,Eunpyeong-gu,1000005,True,Eunpyeong St. Mary's Hospital,14,37.63369,126.9165,10220.0,37.603481,126.929173,31.0,44.0,1.0,1.09,17.00,6.5,874.0
5,Seoul,Seongdong-gu,1000006,True,Seongdong-gu APT,13,37.55713,127.0403,10160.0,37.563277,127.036647,21.0,30.0,2.0,0.97,14.76,5.3,593.0
6,Seoul,Jongno-gu,1000007,True,Jongno Community Center,10,37.57681,127.006,10230.0,37.572999,126.979189,13.0,17.0,3.0,1.71,18.27,6.8,668.0
7,Seoul,Jung-gu,1000008,True,Jung-gu Fashion Company,7,37.562405,126.984377,10240.0,37.563988,126.997530,12.0,14.0,2.0,0.94,18.42,7.4,728.0
8,Seoul,from other city,1000009,True,Shincheonji Church,8,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Seoul,-,1000010,False,overseas inflow,321,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# combind with SQL

In [37]:
cases.registerTempTable("temp")

In [38]:
newDf = sqlContext.sql('select * from temp')

In [39]:
newDf.show()

+-------+--------+---------------+-----+--------------------+---------+---------+----------+
|case_id|province|           city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+---------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|     Yongsan-gu| true|       Itaewon Clubs|       72|37.538621|126.992652|
|1000002|   Seoul|        Guro-gu| true| Guro-gu Call Center|       98|37.508163|126.884387|
|1000003|   Seoul|  Dongdaemun-gu| true|       Dongan Church|       20|37.592888|127.056766|
|1000004|   Seoul|        Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000005|   Seoul|   Eunpyeong-gu| true|Eunpyeong St. Mar...|       14| 37.63369|  126.9165|
|1000006|   Seoul|   Seongdong-gu| true|    Seongdong-gu APT|       13| 37.55713|  127.0403|
|1000007|   Seoul|      Jongno-gu| true|Jongno Community ...|       10| 37.57681|   127.006|
|1000008|   Seoul|        Jung-gu| true|Jung-gu Fashion C...|        7

In [40]:
seoul_info = sqlContext.sql('select * from temp where province="Seoul" ')

In [42]:
seoul_info.show()

+-------+--------+---------------+-----+--------------------+---------+---------+----------+
|case_id|province|           city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+---------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|     Yongsan-gu| true|       Itaewon Clubs|       72|37.538621|126.992652|
|1000002|   Seoul|        Guro-gu| true| Guro-gu Call Center|       98|37.508163|126.884387|
|1000003|   Seoul|  Dongdaemun-gu| true|       Dongan Church|       20|37.592888|127.056766|
|1000004|   Seoul|        Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000005|   Seoul|   Eunpyeong-gu| true|Eunpyeong St. Mar...|       14| 37.63369|  126.9165|
|1000006|   Seoul|   Seongdong-gu| true|    Seongdong-gu APT|       13| 37.55713|  127.0403|
|1000007|   Seoul|      Jongno-gu| true|Jongno Community ...|       10| 37.57681|   127.006|
|1000008|   Seoul|        Jung-gu| true|Jung-gu Fashion C...|        7